In [22]:
import os, sys, glob, shutil
import argparse
import json
import yaml
import numpy as np
from pprint import pprint

import torch
import yaml
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from scipy.io import wavfile
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

In [2]:
%cd "D:\Schoolwork\TERM 3\WORK\visual_prosody"

D:\Schoolwork\TERM 3\WORK\visual_prosody


In [4]:
split_txt_val_path = r'.\preprocessed_data\Ego4D_final_v6\val.txt'
val_uids = []
with open(split_txt_val_path) as file:
    for line in file:
        # print(line.split('|')[0])
        val_uids.append(line.split('|')[0])

In [5]:
print(len(val_uids))

2772


In [23]:
from utils.model import get_model, get_vocoder, get_param_num, vocoder_infer
from utils.tools import to_device, log, synth_one_sample, expand, plot_mel
from model import FastSpeech2Loss
from dataset import Dataset
# from utils.auto_tqdm import tqdm
from evaluate import evaluate

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
parser = argparse.ArgumentParser()
parser.add_argument("--restore_step", type=int, default=0)
parser.add_argument(
    "-p",
    "--preprocess_config",
    type=str,
    required=True,
    help="path to preprocess.yaml",
)
parser.add_argument(
    "-m", "--model_config", type=str, required=True, help="path to model.yaml"
)
parser.add_argument(
    "-t", "--train_config", type=str, required=True, help="path to train.yaml"
)

argString = '-p ./config/Ego4D_final_v6/0703a_preprocess.yaml -m ./config/Ego4D_final_v6/0703a_model.yaml -t ./config/Ego4D_final_v6/0703a_train.yaml'
# args = parser.parse_args()
args = parser.parse_args(argString.split())

In [9]:
pprint(args)
# Read Config
preprocess_config = yaml.load(
    open(args.preprocess_config, "r"), Loader=yaml.FullLoader
)
model_config = yaml.load(open(args.model_config, "r"), Loader=yaml.FullLoader)
train_config = yaml.load(open(args.train_config, "r"), Loader=yaml.FullLoader)
configs = (preprocess_config, model_config, train_config)
print("Prepare training ...")

preprocess_config, model_config, train_config = configs

Namespace(restore_step=0, preprocess_config='./config/Ego4D_final_v6/0703a_preprocess.yaml', model_config='./config/Ego4D_final_v6/0703a_model.yaml', train_config='./config/Ego4D_final_v6/0703a_train.yaml')
Prepare training ...


In [10]:
ckpt_path = r'./output/0703a/ckpt/Ego4D_final_v6/1400000.pth.tar'
ckpt = torch.load(ckpt_path)

In [11]:
# Prepare model
model, optimizer = get_model(args, configs, device, train=True)
model.load_state_dict(ckpt["model"], strict=True)
model.to(device)
model = nn.DataParallel(model)
num_param = get_param_num(model)
Loss = FastSpeech2Loss(preprocess_config, model_config).to(device)
print("Number of FastSpeech2 Parameters:", num_param)

=> Using speaker embeddings.
True
2
=> Using VarianceAdaptorWithSpeaker.
Number of FastSpeech2 Parameters: 35165553


In [12]:
# Load vocoder
vocoder = get_vocoder(model_config, device)
step = args.restore_step + 1
model.eval()
print()
dataset = Dataset(
    "val.txt", 'val', preprocess_config, train_config, sort=False, drop_last=False
)

Removing weight norm...



In [13]:
len(dataset)

2772

In [14]:
batch_size = train_config["optimizer"]["batch_size"]
batch_size = 1

In [15]:
loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=dataset.collate_fn,
    )
# Get loss function
Loss = FastSpeech2Loss(preprocess_config, model_config).to(device)

In [16]:
os.path.join(train_config['path']['result_path'], 'plots')

'./output/0703a/result/Ego4D_final_v6\\plots'

In [17]:
# => batch:
# return (
#     ids,
#     raw_texts,
#     speakers,
#     texts,
#     text_lens,
#     max(text_lens),
#     mels,
#     mel_lens,
#     max(mel_lens),
#     pitches,
#     energies,
#     durations,
#     speaker_embeddings,
# )
# [12] 对应speaker embedding


# => model output / prediction
# return (
#     output,
#     postnet_output,
#     p_predictions,
#     e_predictions,
#     log_d_predictions,
#     d_rounded,
#     src_masks,
#     mel_masks,
#     src_lens,
#     mel_lens,
# )
output_plot_path = os.path.join(train_config['path']['result_path'], 'plot')
output_mel_syn_path = os.path.join(train_config['path']['result_path'], 'mel', 'syn')
output_mel_gt_path = os.path.join(train_config['path']['result_path'], 'mel', 'gt')
output_wav_syn_path = os.path.join(train_config['path']['result_path'], 'wav', 'synthesized')
output_wav_rec_path = os.path.join(train_config['path']['result_path'], 'wav', 'reconstructed')
os.makedirs(output_plot_path, exist_ok=True)
os.makedirs(output_mel_syn_path, exist_ok=True)
os.makedirs(output_mel_gt_path, exist_ok=True)
os.makedirs(output_wav_syn_path, exist_ok=True)
os.makedirs(output_wav_rec_path, exist_ok=True)

In [18]:
for batchs in tqdm(loader):
    for targets in batchs:
        targets = to_device(targets, device)
        with torch.no_grad():
            predictions = model(*(targets[2:]))
        basenames = targets[0]
        for i in range(len(predictions[0])):
            basename = basenames[i]
            src_len = predictions[8][i].item()
            mel_len = predictions[9][i].item()
            mel_prediction = predictions[1][i, :mel_len].detach().transpose(0, 1)
            mel_target = targets[6][i, :mel_len].detach().transpose(0, 1)

            torch.save(mel_prediction.cpu(), os.path.join(output_mel_syn_path, f"{basename}.pt"))
            torch.save(mel_target.cpu(), os.path.join(output_mel_gt_path, f"{basename}.pt"))
            
            
            duration = predictions[5][i, :src_len].detach().cpu().numpy()
            if preprocess_config["preprocessing"]["pitch"]["feature"] == "phoneme_level":
                pitch = predictions[2][i, :src_len].detach().cpu().numpy()
                pitch = expand(pitch, duration)
            else:
                pitch = predictions[2][i, :mel_len].detach().cpu().numpy()
            if preprocess_config["preprocessing"]["energy"]["feature"] == "phoneme_level":
                energy = predictions[3][i, :src_len].detach().cpu().numpy()
                energy = expand(energy, duration)
            else:
                energy = predictions[3][i, :mel_len].detach().cpu().numpy()

            with open(os.path.join(preprocess_config["path"]["preprocessed_path"], 
                                   "stats.json")) as f:
                stats = json.load(f)
                stats = stats["pitch"] + stats["energy"][:2]
                                       
            fig = plot_mel(
                [
                    (mel_prediction.cpu().numpy(), pitch, energy),
                    (mel_target.cpu().numpy(), pitch, energy),
                ],
                stats,
                ["Synthetized Spectrogram", "Ground-Truth Spectrogram"],
            )
            ### TODO: change to svg
            plt.savefig(os.path.join(output_plot_path, f"{basename}.png"))
            plt.close()

        # from .model import vocoder_infer

        mel_predictions = predictions[1].transpose(1, 2)
        mel_targets = targets[6].transpose(1, 2)
        
        lengths = predictions[9] * preprocess_config["preprocessing"]["stft"]["hop_length"]
        wav_predictions = vocoder_infer(
            mel_predictions, vocoder, model_config, preprocess_config, lengths=lengths
        )
        wav_targets = vocoder_infer(
        mel_targets, vocoder, model_config, preprocess_config, lengths=lengths
    )
    
        sampling_rate = preprocess_config["preprocessing"]["audio"]["sampling_rate"]
        for wav, basename in zip(wav_predictions, basenames):
            wavfile.write(os.path.join(output_wav_syn_path, f"{basename}.wav"), sampling_rate, wav)
        for wav, basename in zip(wav_targets, basenames):
            wavfile.write(os.path.join(output_wav_rec_path, f"{basename}.wav"), sampling_rate, wav)

        
    #     break
    # break

  0%|          | 0/2772 [00:00<?, ?it/s]

# MCD

In [24]:
import os, sys, glob, shutil
import os.path as op
import argparse
import json
import yaml
import numpy as np
import pandas as pd
from pprint import pprint
import os.path as op

import torch
import torch.nn.functional as F
import torchaudio
import argparse
import fnmatch
import logging
import multiprocessing as mp
from typing import Dict, List, Tuple

import librosa
import pysptk
import pyworld as pw
import soundfile as sf
from fastdtw import fastdtw
from scipy import spatial

In [20]:
split_txt_val_path = r'.\preprocessed_data\Ego4D_final_v6\val.txt'
val_uids = []
with open(split_txt_val_path) as file:
    for line in file:
        # print(line.split('|')[0])
        val_uids.append(line.split('|')[0])

In [21]:
len(val_uids)

2772

In [22]:
recon_folder = r'.\output\0703a\result\Ego4D_final_v6\wav\reconstructed'
syn_folder = r'.\output\0703a\result\Ego4D_final_v6\wav\synthesized'
gt_folder = r'.\raw_data\Ego4D_final_v6\val\Ego4D_final_v6'

In [23]:
sr = 22050
melkwargs = {
    "n_fft": int(0.05 * sr), "win_length": int(0.05 * sr),
    "hop_length": int(0.0125 * sr), "f_min": 20,
    "n_mels": 80, "window_fn": torch.hann_window
}
mfcc_fn = torchaudio.transforms.MFCC(
    sr, n_mfcc=13, log_mels=True, melkwargs=melkwargs
)

In [24]:
%%time
uids = []
MCDs_recon = []
for uid in tqdm(val_uids):
    uids.append(uid)
    wav_1, sr = torchaudio.load(op.join(recon_folder, f"{uid}.wav"))
    wav_2, sr = torchaudio.load(op.join(syn_folder, f"{uid}.wav"))
    wav_1 = wav_1.squeeze()
    wav_2 = wav_2.squeeze()
    mel_1 = mfcc_fn(wav_1).T.numpy()
    mel_2 = mfcc_fn(wav_2).T.numpy()
    # DTW
    _, path = fastdtw(mel_2, mel_1, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    mel_2 = mel_2[twf[0]]
    mel_1 = mel_1[twf[1]]
    # We sum the squared differences over the first K MFCCs, skipping ct,0
    mel_1 = mel_1[:, 1:]
    mel_2 = mel_2[:, 1:]
    result = (((mel_1 - mel_2) ** 2).sum(axis=1)**0.5).mean()
    MCDs_recon.append(result)

  0%|          | 0/2772 [00:00<?, ?it/s]

CPU times: total: 7min 58s
Wall time: 2min 9s


In [25]:
%%time
uids = []
MCDs_gt = []
for uid in tqdm(val_uids):
    uids.append(uid)
    wav_1, sr = torchaudio.load(op.join(gt_folder, f"{uid}.wav"))
    wav_2, sr = torchaudio.load(op.join(syn_folder, f"{uid}.wav"))
    wav_1 = wav_1.squeeze()
    wav_2 = wav_2.squeeze()
    mel_1 = mfcc_fn(wav_1).T.numpy()
    mel_2 = mfcc_fn(wav_2).T.numpy()
    # DTW
    _, path = fastdtw(mel_2, mel_1, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    mel_2 = mel_2[twf[0]]
    mel_1 = mel_1[twf[1]]
    # We sum the squared differences over the first K MFCCs, skipping ct,0
    mel_1 = mel_1[:, 1:]
    mel_2 = mel_2[:, 1:]
    result = (((mel_1 - mel_2) ** 2).sum(axis=1)**0.5).mean()
    MCDs_gt.append(result)

  0%|          | 0/2772 [00:00<?, ?it/s]

CPU times: total: 8min 35s
Wall time: 2min 22s


In [26]:
v6_mcd_df = pd.DataFrame({
    'uid': uids,
    'MCD_recon': MCDs_recon,
    'MCD_gt': MCDs_gt,
})

In [27]:
v6_mcd_df.to_csv(r".\jupyter_walkthrough\metrics\MCD_0703a_1M4.csv")

In [28]:
print(f"MCDs_recon mean on 0703a_1M4: {torch.tensor(MCDs_recon).mean()}")
print(f"MCDs_recon std on 0703a_1M4: {torch.tensor(MCDs_recon).std()}")

print(f"MCDs_gt mean on 0703a_1M4: {torch.tensor(MCDs_gt).mean()}")
print(f"MCDs_gt std on 0703a_1M4: {torch.tensor(MCDs_gt).std()}")

MCDs_recon mean on 0703a_1M4: 12.921546936035156
MCDs_recon std on 0703a_1M4: 2.466970205307007
MCDs_gt mean on 0703a_1M4: 13.955644607543945
MCDs_gt std on 0703a_1M4: 2.486111879348755


In [29]:
v6_mcd_df.nsmallest(5, 'MCD_recon')

,uid,MCD_recon,MCD_gt
29,0fc4a693-2ba5-419b-83cc-51d5e70b75b9,5.854140,7.754389
562,dcd3a1bc-beb5-49ac-a2c0-833ff646620f,6.501545,14.302723
1557,7f53c386-bf80-47bd-8dbe-295b21eeb289,6.755201,8.376004
109,3247ffa3-adc8-4978-9be8-4b07a0995e7b,7.198318,8.091181
580,b8adbe87-2c00-411b-8e12-d769fb3ee849,7.651339,9.147169


In [30]:
v6_mcd_df.nlargest(5, 'MCD_recon')

,uid,MCD_recon,MCD_gt
1189,2bf14b91-b7cd-4ccc-b21a-020b2c0aa3c3,26.086525,26.590265
293,b97e171a-23f7-4c73-94a3-c2fd18d8f340,25.599823,25.150154
465,927dcb5d-c33d-42f2-9008-73c355853323,25.436575,26.151859
2242,3a2dd434-33a0-498c-9947-df56e604dd61,25.207720,30.353777
2129,4c848686-910e-41bd-b183-9a8867d236a9,24.893915,26.048285


# inference on train

In [3]:
from utils.model import get_model, get_vocoder, get_param_num, vocoder_infer
from utils.tools import to_device, log, synth_one_sample, expand, plot_mel
from model import FastSpeech2Loss
from dataset import Dataset
# from utils.auto_tqdm import tqdm
from evaluate import evaluate

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
parser = argparse.ArgumentParser()
parser.add_argument("--restore_step", type=int, default=0)
parser.add_argument(
    "-p",
    "--preprocess_config",
    type=str,
    required=True,
    help="path to preprocess.yaml",
)
parser.add_argument(
    "-m", "--model_config", type=str, required=True, help="path to model.yaml"
)
parser.add_argument(
    "-t", "--train_config", type=str, required=True, help="path to train.yaml"
)

argString = '-p ./config/Ego4D_final_v6/0703a_preprocess.yaml -m ./config/Ego4D_final_v6/0703a_model.yaml -t ./config/Ego4D_final_v6/0703a_train.yaml'
# args = parser.parse_args()
args = parser.parse_args(argString.split())

In [6]:
pprint(args)
# Read Config
preprocess_config = yaml.load(
    open(args.preprocess_config, "r"), Loader=yaml.FullLoader
)
model_config = yaml.load(open(args.model_config, "r"), Loader=yaml.FullLoader)
train_config = yaml.load(open(args.train_config, "r"), Loader=yaml.FullLoader)
configs = (preprocess_config, model_config, train_config)
print("Prepare training ...")

preprocess_config, model_config, train_config = configs

Namespace(restore_step=0, preprocess_config='./config/Ego4D_final_v6/0703a_preprocess.yaml', model_config='./config/Ego4D_final_v6/0703a_model.yaml', train_config='./config/Ego4D_final_v6/0703a_train.yaml')
Prepare training ...


In [7]:
ckpt_path = r'./output/0703a/ckpt/Ego4D_final_v6/1400000.pth.tar'
ckpt = torch.load(ckpt_path)

In [8]:
# Prepare model
model, optimizer = get_model(args, configs, device, train=True)
model.load_state_dict(ckpt["model"], strict=True)
model.to(device)
model = nn.DataParallel(model)
num_param = get_param_num(model)
Loss = FastSpeech2Loss(preprocess_config, model_config).to(device)
print("Number of FastSpeech2 Parameters:", num_param)

=> Using speaker embeddings.
True
2
=> Using VarianceAdaptorWithSpeaker.
Number of FastSpeech2 Parameters: 35165553


In [9]:
# Load vocoder
vocoder = get_vocoder(model_config, device)
step = args.restore_step + 1
model.eval()
print()
dataset = Dataset(
    "train.txt", 'train', preprocess_config, train_config, sort=False, drop_last=False
)

Removing weight norm...



In [10]:
len(dataset)

27292

In [11]:
batch_size = train_config["optimizer"]["batch_size"]
batch_size = 1

In [12]:
loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=dataset.collate_fn,
    )
# Get loss function
Loss = FastSpeech2Loss(preprocess_config, model_config).to(device)

In [13]:
# => batch:
# return (
#     ids,
#     raw_texts,
#     speakers,
#     texts,
#     text_lens,
#     max(text_lens),
#     mels,
#     mel_lens,
#     max(mel_lens),
#     pitches,
#     energies,
#     durations,
#     speaker_embeddings,
# )
# [12] 对应speaker embedding


# => model output / prediction
# return (
#     output,
#     postnet_output,
#     p_predictions,
#     e_predictions,
#     log_d_predictions,
#     d_rounded,
#     src_masks,
#     mel_masks,
#     src_lens,
#     mel_lens,
# )
output_plot_path = os.path.join(train_config['path']['result_path'], 'train', 'plot')
output_mel_syn_path = os.path.join(train_config['path']['result_path'], 'train', 'mel', 'syn')
output_mel_gt_path = os.path.join(train_config['path']['result_path'], 'train', 'mel', 'gt')
output_wav_syn_path = os.path.join(train_config['path']['result_path'], 'train', 'wav', 'synthesized')
output_wav_rec_path = os.path.join(train_config['path']['result_path'], 'train', 'wav', 'reconstructed')
os.makedirs(output_plot_path, exist_ok=True)
os.makedirs(output_mel_syn_path, exist_ok=True)
os.makedirs(output_mel_gt_path, exist_ok=True)
os.makedirs(output_wav_syn_path, exist_ok=True)
os.makedirs(output_wav_rec_path, exist_ok=True)

In [14]:
for batchs in tqdm(loader):
    for targets in batchs:
        targets = to_device(targets, device)
        with torch.no_grad():
            predictions = model(*(targets[2:]))
        basenames = targets[0]
        for i in range(len(predictions[0])):
            basename = basenames[i]
            src_len = predictions[8][i].item()
            mel_len = predictions[9][i].item()
            mel_prediction = predictions[1][i, :mel_len].detach().transpose(0, 1)
            mel_target = targets[6][i, :mel_len].detach().transpose(0, 1)

            torch.save(mel_prediction.cpu(), os.path.join(output_mel_syn_path, f"{basename}.pt"))
            torch.save(mel_target.cpu(), os.path.join(output_mel_gt_path, f"{basename}.pt"))
            
            
            duration = predictions[5][i, :src_len].detach().cpu().numpy()
            if preprocess_config["preprocessing"]["pitch"]["feature"] == "phoneme_level":
                pitch = predictions[2][i, :src_len].detach().cpu().numpy()
                pitch = expand(pitch, duration)
            else:
                pitch = predictions[2][i, :mel_len].detach().cpu().numpy()
            if preprocess_config["preprocessing"]["energy"]["feature"] == "phoneme_level":
                energy = predictions[3][i, :src_len].detach().cpu().numpy()
                energy = expand(energy, duration)
            else:
                energy = predictions[3][i, :mel_len].detach().cpu().numpy()

            with open(os.path.join(preprocess_config["path"]["preprocessed_path"], 
                                   "stats.json")) as f:
                stats = json.load(f)
                stats = stats["pitch"] + stats["energy"][:2]
                                       
            fig = plot_mel(
                [
                    (mel_prediction.cpu().numpy(), pitch, energy),
                    (mel_target.cpu().numpy(), pitch, energy),
                ],
                stats,
                ["Synthetized Spectrogram", "Ground-Truth Spectrogram"],
            )
            ### TODO: change to svg
            plt.savefig(os.path.join(output_plot_path, f"{basename}.png"))
            plt.close()

        # from .model import vocoder_infer

        mel_predictions = predictions[1].transpose(1, 2)
        mel_targets = targets[6].transpose(1, 2)
        
        lengths = predictions[9] * preprocess_config["preprocessing"]["stft"]["hop_length"]
        wav_predictions = vocoder_infer(
            mel_predictions, vocoder, model_config, preprocess_config, lengths=lengths
        )
        wav_targets = vocoder_infer(
        mel_targets, vocoder, model_config, preprocess_config, lengths=lengths
    )
    
        sampling_rate = preprocess_config["preprocessing"]["audio"]["sampling_rate"]
        for wav, basename in zip(wav_predictions, basenames):
            wavfile.write(os.path.join(output_wav_syn_path, f"{basename}.wav"), sampling_rate, wav)
        for wav, basename in zip(wav_targets, basenames):
            wavfile.write(os.path.join(output_wav_rec_path, f"{basename}.wav"), sampling_rate, wav)

        
    #     break
    # break

  0%|          | 0/27292 [00:00<?, ?it/s]

In [15]:
print(11111)

11111


# MCD on train

In [25]:
split_txt_train_path = r'.\preprocessed_data\Ego4D_final_v6\train.txt'
train_uids = []
with open(split_txt_train_path) as file:
    for line in file:
        # print(line.split('|')[0])
        train_uids.append(line.split('|')[0])

In [26]:
len(train_uids)

27292

In [27]:
recon_folder = r'.\output\0703a\result\Ego4D_final_v6\train\wav\reconstructed'
syn_folder = r'.\output\0703a\result\Ego4D_final_v6\train\wav\synthesized'
gt_folder = r'.\raw_data\Ego4D_final_v6\train\Ego4D_final_v6'

In [28]:
sr = 22050
melkwargs = {
    "n_fft": int(0.05 * sr), "win_length": int(0.05 * sr),
    "hop_length": int(0.0125 * sr), "f_min": 20,
    "n_mels": 80, "window_fn": torch.hann_window
}
mfcc_fn = torchaudio.transforms.MFCC(
    sr, n_mfcc=13, log_mels=True, melkwargs=melkwargs
)

In [29]:
%%time
uids = []
MCDs_recon = []
for uid in tqdm(train_uids):
    uids.append(uid)
    wav_1, sr = torchaudio.load(op.join(recon_folder, f"{uid}.wav"))
    wav_2, sr = torchaudio.load(op.join(syn_folder, f"{uid}.wav"))
    wav_1 = wav_1.squeeze()
    wav_2 = wav_2.squeeze()
    mel_1 = mfcc_fn(wav_1).T.numpy()
    mel_2 = mfcc_fn(wav_2).T.numpy()
    # DTW
    _, path = fastdtw(mel_2, mel_1, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    mel_2 = mel_2[twf[0]]
    mel_1 = mel_1[twf[1]]
    # We sum the squared differences over the first K MFCCs, skipping ct,0
    mel_1 = mel_1[:, 1:]
    mel_2 = mel_2[:, 1:]
    result = (((mel_1 - mel_2) ** 2).sum(axis=1)**0.5).mean()
    MCDs_recon.append(result)

  0%|          | 0/27292 [00:00<?, ?it/s]

CPU times: total: 2h 31min 17s
Wall time: 26min 27s


In [30]:
%%time
uids = []
MCDs_gt = []
for uid in tqdm(train_uids):
    uids.append(uid)
    wav_1, sr = torchaudio.load(op.join(gt_folder, f"{uid}.wav"))
    wav_2, sr = torchaudio.load(op.join(syn_folder, f"{uid}.wav"))
    wav_1 = wav_1.squeeze()
    wav_2 = wav_2.squeeze()
    mel_1 = mfcc_fn(wav_1).T.numpy()
    mel_2 = mfcc_fn(wav_2).T.numpy()
    # DTW
    _, path = fastdtw(mel_2, mel_1, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    mel_2 = mel_2[twf[0]]
    mel_1 = mel_1[twf[1]]
    # We sum the squared differences over the first K MFCCs, skipping ct,0
    mel_1 = mel_1[:, 1:]
    mel_2 = mel_2[:, 1:]
    result = (((mel_1 - mel_2) ** 2).sum(axis=1)**0.5).mean()
    MCDs_gt.append(result)

  0%|          | 0/27292 [00:00<?, ?it/s]

CPU times: total: 2h 54min
Wall time: 30min 36s


In [31]:
v6_mcd_df = pd.DataFrame({
    'uid': uids,
    'MCD_recon': MCDs_recon,
    'MCD_gt': MCDs_gt,
})

In [32]:
v6_mcd_df.to_csv(r".\jupyter_walkthrough\metrics\MCD_0703a_1M4_train.csv")

In [33]:
print(f"MCDs_recon mean on 0703a_1M4_train: {torch.tensor(MCDs_recon).mean()}")
print(f"MCDs_recon std on 0703a_1M4_train: {torch.tensor(MCDs_recon).std()}")

print(f"MCDs_gt mean on 0703a_1M4_train: {torch.tensor(MCDs_gt).mean()}")
print(f"MCDs_gt std on 0703a_1M4_train: {torch.tensor(MCDs_gt).std()}")

MCDs_recon mean on 0703a_1M4_train: 7.545382976531982
MCDs_recon std on 0703a_1M4_train: 1.1076444387435913
MCDs_gt mean on 0703a_1M4_train: 10.29171085357666
MCDs_gt std on 0703a_1M4_train: 2.1469104290008545


In [34]:
v6_mcd_df.nsmallest(5, 'MCD_recon')

,uid,MCD_recon,MCD_gt
15928,435ea1f3-6eff-40a7-93a8-000f4861dc84,3.558908,7.061443
2623,e6d67d17-93c8-4bcf-90ea-26eba9826044,4.070715,6.730083
5551,860c7982-da84-4be9-8c48-b5325cd60c7b,4.110170,7.300344
14280,bcc5cd11-c660-456a-aa17-d9642cf8f800,4.123337,17.850710
7378,514982f0-0b63-490b-ba77-8ba35a8f7d0c,4.176795,10.757712


In [35]:
v6_mcd_df.nlargest(5, 'MCD_recon')

,uid,MCD_recon,MCD_gt
9840,bb12f5eb-67d7-4aa0-941b-b9e8f7bba3c4,16.314287,15.924042
1112,55da5227-d472-473a-a38b-a79f56af1448,15.414665,16.089445
5318,b3e2c12d-8530-41f7-877b-076dd450e5a1,15.108876,18.417839
22978,c6d71379-d3c0-4e5c-b835-87423128767a,15.070946,14.154649
16678,8d687ac9-1b7e-49f0-be0d-f6ca953ae7ea,14.459346,15.932846
